In [1]:
import sys
sys.path.append('../input/timm-package/pytorch-image-models-master')

import os
import cv2
import timm
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.backends.cudnn as cudnn

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90, ElasticTransform,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, GaussianBlur, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,
    ShiftScaleRotate, CenterCrop, Resize, GridDropout
)
from albumentations.pytorch import ToTensorV2

cfg = {
    'model_arch': 'resnet200d',
    'img_size': 512,
    'batch_size': 16,
    'target_size': 11,
    'target_cols': ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                    'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                    'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                    'Swan Ganz Catheter Present'],
    'device': 'cuda',
}
device = cfg['device']

data_path = os.path.abspath(os.path.join(os.path.pardir, 'input/ranzcr-clip-catheter-line-classification'))
train_path = os.path.join(data_path, 'train')
label_path = os.path.join(data_path, 'train.csv')
test_path = os.path.join(data_path, 'test')
csv_path = os.path.join(data_path, 'sample_submission.csv')
cwd = os.path.abspath('.')
cudnn.benchmark = True

def get_img(img_path):
    img_bgr = cv2.imread(img_path)
    return img_bgr[..., ::-1]

def get_valid_transforms(img_size=700):
    return Compose([
            Resize(img_size, img_size),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(p=1.0),
        ], p=1.0)

class CatheterDataset(Dataset):
    def __init__(self, df, dPath=None, transforms=None):
        super(CatheterDataset, self).__init__()
        self.df = df.reset_index(drop=True)
        self.dPath = dPath
        self.transforms = transforms

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        uid = self.df['StudyInstanceUID'].values[idx]
        img_name = uid + '.jpg'
        img_path = os.path.join(self.dPath, img_name)
        image = get_img(img_path)
        augmented = self.transforms(image=image)
        img = augmented['image']
        return img
    
class CustomResNet200D(nn.Module):
    def __init__(self, model_name=cfg['model_arch']):
        super().__init__()
        self.model = timm.create_model(model_name)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        features = self.model(x)
        pooled_features = self.pooling(features).view(x.size(0), -1)
        output = self.fc(pooled_features)
        return output
    
class CustomInceptionv3(nn.Module):
    def __init__(self, model_name='inception_v3'):
        super().__init__()
        self.model = timm.create_model(model_name)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        features = self.model(x)
        pooled_features = self.pooling(features).view(x.size(0), -1)
        output = self.fc(pooled_features)
        return output
    
class CustomEffb5(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b5'):
        super().__init__()
        self.model = timm.create_model(model_name)
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        features = self.model(x)
        pooled_features = self.pooling(features).view(x.size(0), -1)
        output = self.fc(pooled_features)
        return output

def inference(test_loader, nets):
    progress_bar = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for net in nets:
        net.eval()
        net.to(device)

    for batch_idx, images in progress_bar:
        images = images.to(device)
        avg_preds = []
        for net in nets:
            with torch.no_grad():
                y_preds1 = net(images)
                y_preds2 = net(images.flip(-1))
            y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy()) / 2
            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

def submission():
    torch.cuda.empty_cache()
    predictions = []
    
    weights = {
        # resnet200d
        384: 0.12,
        512: 0.28,
        700: 0.36,
        # inception-v3
        640: 0.12
    }
    
    for dire, img_size in zip(('../input/size384', '../input/resnet200d-stage3', '../input/size700'), (384, 512, 700)):
        test_df = pd.read_csv(csv_path)
        test_set = CatheterDataset(test_df, test_path, get_valid_transforms(img_size=img_size))
        test_loader = DataLoader(test_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=12)
        nets = []
        for name in os.listdir(dire):
            net = CustomResNet200D()
            net = torch.nn.DataParallel(net)
            checkpoint = torch.load(os.path.join(dire, name))
            net.load_state_dict(checkpoint['net'])
            net.eval()
            nets.append(net)
        p = inference(test_loader, nets)
        predictions.append(weights[img_size] * p)
        del test_loader, nets
        torch.cuda.empty_cache()
        
    for dire, img_size in zip(('../input/inception640',), (640,)):
        test_df = pd.read_csv(csv_path)
        test_set = CatheterDataset(test_df, test_path, get_valid_transforms(img_size=img_size))
        test_loader = DataLoader(test_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=12)
        nets = []
        cfg['model_arch'] = 'inception_v3'
        for name in os.listdir(dire):
            net = CustomInceptionv3()
            net = torch.nn.DataParallel(net, device_ids=[0])
            checkpoint = torch.load(os.path.join(dire, name))
            net.load_state_dict(checkpoint['net'])
            net.eval()
            nets.append(net)
        p = inference(test_loader, nets)
        predictions.append(weights[img_size] * p)
        del test_loader, nets
        torch.cuda.empty_cache()
    
    for dire, img_size in zip(('../input/effnetb5',), (640,)):
        test_df = pd.read_csv(csv_path)
        test_set = CatheterDataset(test_df, test_path, get_valid_transforms(img_size=img_size))
        test_loader = DataLoader(test_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=12)
        nets = []
        for name in os.listdir(dire):
            net = CustomEffb5()
            net = torch.nn.DataParallel(net, device_ids=[0])
            checkpoint = torch.load(os.path.join(dire, name))
            net.load_state_dict(checkpoint['net'])
            net.eval()
            nets.append(net)
        p = inference(test_loader, nets)
        weight = 0.12
        predictions.append(weight * p)
        del test_loader, nets
        torch.cuda.empty_cache()
        
    predictions = np.sum(predictions, axis=0)
    test_df[cfg['target_cols']] = predictions
    test_df[['StudyInstanceUID'] + cfg['target_cols']].to_csv('submission.csv', index=False)
    return test_df

# df = submission()
# df
# for name in os.listdir('../input/size384'):
#     checkpoint = torch.load(f'../input/size384/{name}')
#     print(name, checkpoint['auc'])
# for name in os.listdir('../input/inception640'):
#     checkpoint = torch.load('../input/inception640/' + name)
#     print(name, checkpoint['auc'])

In [3]:
directory = '../input/inception640/'
for name in os.listdir(directory):
    checkpoint = torch.load(directory + name)
    print(name, checkpoint['auc'])

catheter-inception_v3-student-fold1-auc.pth 0.9505887035976209
catheter-inception_v3-student-fold0-auc.pth 0.9508372489830479
catheter-inception_v3-student-fold2-auc.pth 0.9485751370863545
catheter-inception_v3-student-fold4-auc.pth 0.9519996432056795
catheter-inception_v3-student-fold3-auc.pth 0.9510773307614163


In [4]:
directory = '../input/effnetb5/'
for name in os.listdir(directory):
    checkpoint = torch.load(directory + name)
    print(name, checkpoint['auc'])

catheter-tf_efficientnet_b5-student-fold1-auc.pth 0.9543205279911398
catheter-tf_efficientnet_b5-student-fold3-auc.pth 0.9468107697607795
catheter-tf_efficientnet_b5-student-fold4-auc.pth 0.9458153062659285
catheter-tf_efficientnet_b5-student-fold0-auc.pth 0.9572916930852879
catheter-tf_efficientnet_b5-student-fold2-auc.pth 0.951481841674983


In [5]:
directory = '../input/size384/'
for name in os.listdir(directory):
    checkpoint = torch.load(directory + name)
    print(name, checkpoint['auc'])

catheter-resnet200d-student-fold3-auc.pth 0.9526698534855853
catheter-resnet200d-student-fold1-auc.pth 0.9551442535065647
catheter-resnet200d-student-fold0-auc.pth 0.9537666920863934
catheter-resnet200d-student-fold2-auc.pth 0.9502105538167304
catheter-resnet200d-student-fold4-auc.pth 0.95514839786041


In [6]:
directory = '../input/size700/'
for name in os.listdir(directory):
    checkpoint = torch.load(directory + name)
    print(name, checkpoint['auc'])

catheter-resnet200d-student-fold2-700-auc.pth 0.9615204393214424
catheter-resnet200d-student-fold3-700-auc.pth 0.964261895294204
catheter-resnet200d-student-fold4-700-auc.pth 0.9659442361418951
catheter-resnet200d-student-fold0-700-auc.pth 0.9651744732864311
catheter-resnet200d-student-fold1-700-auc.pth 0.9640389998484064


In [ ]:
directory = '../input/resnet200d-stage3/'
for name in os.listdir(directory):
    checkpoint = torch.load(directory + name)
    print(name, checkpoint['auc'])

In [ ]:
auc_l = [0.9505887035976209
 0.9508372489830479
, 0.9485751370863545
, 0.9519996432056795
, 0.9510773307614163]